In [1]:
from esfMRI import sliceWindows, joint_cluster_save_states, plot_sates, align, clustering_evaluate, windows_evaluate, step_evaluate, AIC, BIC, plot_evaluated
from sklearn import cluster, metrics
from nilearn import connectome
import numpy as np
import pickle
import json
import math
import os

In [2]:
# 可调节参数
window_length_Second = [60, 100, 120] # 窗口尺寸，单位s
sliding_step = 1 # 滑动步长，单位s
target_states = [3, 4, 5, 6] # 目标状态数

### 猜测聚类簇数

1. 肘点法：绘制inertia随k值变化的曲线，转折幅度最大的点作为簇数。

### 评估聚类质量

对于不存在已知分类的评价，只能采用内部评价指标  
基础参数有
1. 紧密度（Compactness）
2. 分割度（Seperation）
3. 误差平方和（SSE: Sum of squares of errors）

评价指标
1. 轮廓系数 —— 越大越好
2. Calinski-Harabasz Index（CH） —— 越大越好
3. Davies-Bouldin Index（DB） —— 越小越好

### 模型评估方法

1. 贝叶斯信息准则（BIC） —— 越小越好
2. 赤池信息量准则（AIC）

In [3]:
# 导入时间序列
with open("time_series2.pkl", "rb") as f:
    data = pickle.load(f)

In [18]:
# 拼接后聚类评估窗口尺寸影响
for subid in data:
    for k in target_states:
        save_dir = f"./cluster_evaluate/window_length/joint/{subid}"
        os.makedirs(save_dir, exist_ok=True)
        windows_evaluate(data, subid, range(30, 180, 10), 1, k, f"{save_dir}/{k}_states.png")

In [4]:
# 评估窗口尺寸，全部数据拼接
for k in target_states:
    inertias = []
    scs = []
    chs = []
    dbs = []
    aic = []
    bic = []
    for time in range(30, 180, 10):
        windows = []
        for subid in data:
            for run, items in data[subid]["ses-preop"].items():
                preopTR = math.ceil(time/items["TR"])
                windows += sliceWindows(items["time_series"], preopTR, 2)
            for run, items in data[subid]["ses-postop"].items():
                postopTR = math.ceil(time/items["TR"])
                windows += sliceWindows(items["time_series"], postopTR, 2)
        fcs = connectome.ConnectivityMeasure(kind="correlation").fit_transform(windows)
        del windows
        fcs = fcs.reshape((fcs.shape[0], 13456))
        if k < fcs.shape[0]:
            center, states, inertia = cluster.k_means(fcs, k)
            inertias.append(inertia) # 肘点法
            scs.append(metrics.silhouette_score(fcs, states)) # 轮廓系数
            chs.append(metrics.calinski_harabasz_score(fcs, states)) # CH，方差比
            dbs.append(metrics.davies_bouldin_score(fcs, states)) # DB
            aic.append(AIC(fcs.shape[0], k, inertia))
            bic.append(BIC(fcs.shape[0], k, inertia))
        else:
            inertias.append(inertias[-1])
            scs.append(scs[-1])
            chs.append(chs[-1])
            dbs.append(dbs[-1])
            aic.append(aic[-1])
            bic.append(bic[-1])
        del fcs
    # 绘图
    plot_evaluated(range(30, 180, 10), inertias=inertias, scs=scs, chs=chs, dbs=dbs, aic=aic, bic=bic, save_path=f"cluster_evaluate/window_length/total/{k}_states.png")

In [ ]:
# 拼接后聚类评估步长影响
for subid in data:
    for time in window_length_Second:
        # if time != 150:
        #     continue
        for k in target_states:
            save_dir = f"cluster_evaluate/joint/{subid}/{k}_states"
            os.makedirs(save_dir, exist_ok=True)
            step_evaluate(time, range(1, 10), k, save_dir, data[subid]["ses-preop"], data[subid]["ses-postop"])

In [4]:
# 全部拼接后聚类评估步长影响
time_series_preop = {}
time_series_postop = {}
for subid in data:
    for run, items in data[subid]["ses-preop"].items():
        time_series_preop[f"{subid}-{run}"] = items
    for run, items in data[subid]["ses-postop"].items():
        time_series_postop[f"{subid}-{run}"] = items
step_evaluate(120, range(1, 10), 3, "cluster_evaluate/step", time_series_preop, time_series_postop)

In [4]:
# 拼接后聚类评估
for subid in data:
    for time in window_length_Second:
        # if time != 150:
        #     continue
        windows_preop = []
        windows_postop = []
        for run, items in data[subid]["ses-preop"].items():
            intervalFrame = math.ceil(sliding_step/items["TR"])
            preopFrame = math.ceil(time/items["TR"])
            tmp = sliceWindows(items["time_series"], preopFrame, intervalFrame)
            windows_preop += tmp
        for run, items in data[subid]["ses-postop"].items():
            intervalFrame = math.ceil(sliding_step/items["TR"])
            postopFrame = math.ceil(time/items["TR"])
            tmp = sliceWindows(items["time_series"], postopFrame, intervalFrame)
            windows_postop += tmp
        save_dir = f"cluster_evaluate/joint/{subid}/"
        os.makedirs(save_dir, exist_ok=True)
        clustering_evaluate(windows_preop, range(2, 15), f"{save_dir}/preop_{time}.png")
        clustering_evaluate(windows_postop, range(2, 15), f"{save_dir}/postop_{time}.png")
        clustering_evaluate(windows_preop+windows_postop, range(2, 15), f"{save_dir}/total_{time}.png")

In [ ]:
# 每个被试分别聚类，输出状态变化
for subid in data:
    states = {}
    for time in window_length_Second:
        # if time != 150:
        #     continue
        states[time] = {}
        preopFrame = math.ceil(time/data[subid]["ses-preop"]["run-01"]["TR"])
        postopFrame = math.ceil(time/data[subid]["ses-postop"]["run-01"]["TR"])
        windows_preop = []
        windows_postop = []
        window_length_preop = []
        window_length_postop = []
        for run, items in data[subid]["ses-preop"].items():
            tmp = sliceWindows(items["time_series"], preopFrame, 1)
            window_length_preop.append(len(tmp))
            windows_preop += tmp
        for run, items in data[subid]["ses-postop"].items():
            tmp = sliceWindows(items["time_series"], postopFrame, 1)
            window_length_postop.append(len(tmp))
            windows_postop += tmp
        states[time]["length"] = [window_length_preop, window_length_postop]
        for k in target_states:
            states_k = joint_cluster_save_states(windows_preop, windows_postop, k)
            states[time]["preop"] = states_k[0]
            states[time]["postop"] = states_k[1]
            save_dir = f"states_pkl/joint/{subid}"
            os.makedirs(save_dir, exist_ok=True)
            with open(f"{save_dir}/{k}states.pkl", "wb") as f:
                pickle.dump(states, f)

            save_dir = f"states/joint/{k}states/{time}/{subid}/"
            os.makedirs(save_dir, exist_ok=True)
            plot_sates(states[time]["preop"], f"{save_dir}/preop.png")
            plot_sates(states[time]["postop"], f"{save_dir}/postop.png")
            tmp = 0
            for run,length in enumerate(states[time]["length"][0]):
                plot_sates(states[time]["preop"][tmp:tmp+length], f"{save_dir}/preop_run{run+1:0>2d}.png")
                tmp += length
            tmp = 0
            for run,length in enumerate(states[time]["length"][1]):
                plot_sates(states[time]["postop"][tmp:tmp+length], f"{save_dir}/postop_run{run+1:0>2d}.png")
                tmp += length

In [5]:
# 所有人平均后聚类
# 对齐帧数
align_length_preop = 130
align_length_postop = 200

time_series_preop = None
time_series_postop = None
count_preop = 0
count_postop = 0
for subid in data:
    for run, items in data[subid]["ses-preop"].items():
        if items["time_series"].shape[0] < align_length_preop:
            continue
        count_preop += 1
        time_series_preop = align(items["time_series"], align_length_preop) if time_series_preop is None else time_series_preop + align(items["time_series"], align_length_preop)
    for run, items in data[subid]["ses-postop"].items():
        if items["time_series"].shape[0] < align_length_postop:
            continue
        count_postop += 1
        time_series_postop = align(items["time_series"], align_length_postop) if time_series_postop is None else time_series_postop + align(items["time_series"], align_length_postop)
time_series_preop = time_series_preop/count_preop
time_series_postop = time_series_postop/count_postop

In [17]:
# 全体数据平均后，评估窗口尺寸影响
inertias = []
scs = []
chs = []
dbs = []
aic = []
bic = []
for time in range(30, 180, 10):
    windows_preop = sliceWindows(time_series_preop, math.ceil(time/2.26), 1)
    windows_postop = sliceWindows(time_series_postop, math.ceil(time/3), 1)
    windows = windows_preop + windows_postop
    fcs = connectome.ConnectivityMeasure(kind="correlation").fit_transform(windows)
    fcs = fcs.reshape((fcs.shape[0], 13456))
    if k < fcs.shape[0]:
        center, states, inertia = cluster.k_means(fcs, k)
        inertias.append(inertia) # 肘点法
        scs.append(metrics.silhouette_score(fcs, states)) # 轮廓系数
        chs.append(metrics.calinski_harabasz_score(fcs, states)) # CH，方差比
        dbs.append(metrics.davies_bouldin_score(fcs, states)) # DB
        aic.append(AIC(fcs.shape[0], k, inertia))
        bic.append(BIC(fcs.shape[0], k, inertia))
    else:
        inertias.append(inertias[-1])
        scs.append(scs[-1])
        chs.append(chs[-1])
        dbs.append(dbs[-1])
        aic.append(aic[-1])
        bic.append(bic[-1])
plot_evaluated(range(30, 180, 10), inertias=inertias, scs=scs, chs=chs, dbs=dbs, aic=aic, bic=bic, save_path="cluster_evaluate/window_length/average.png")

In [10]:
# 全体数据平均，绘制状态变化
states = {}
for time in window_length_Second:
    states[time] = {}
    preopTR = math.ceil(time/data[subid]["ses-preop"]["run-01"]["TR"])
    postopTR = math.ceil(time/data[subid]["ses-postop"]["run-01"]["TR"])
    windows_preop = sliceWindows(time_series_preop, preopTR, 1)
    windows_postop = sliceWindows(time_series_postop, postopTR, 1)
    for k in target_states:
        if len(windows_preop) <= k and len(windows_postop) <= k:
            continue
        states_k = joint_cluster_save_states(windows_preop, windows_postop, k)
        states[time]["preop"] = states_k[0]
        states[time]["postop"] = states_k[1]
        save_dir = f"states_pkl/average/"
        os.makedirs(save_dir, exist_ok=True)
        with open(f"{save_dir}/{k}states.pkl", "wb") as f:
            pickle.dump(states, f)

        save_dir = f"states/average/{k}states/{time}/"
        os.makedirs(save_dir, exist_ok=True)
        plot_sates(states[time]["preop"], k, f"{save_dir}/preop.png")
        plot_sates(states[time]["postop"], k, f"{save_dir}/postop.png")

In [3]:
with open("slidingWindows.pkl", "rb") as f:
    slidingWindows = pickle.load(f)

In [4]:
# 拼接后聚类评估
windows_preop = []
windows_postop = []
save_path = "cluster_evaluate/total"
os.makedirs(save_path, exist_ok=True)
time = 180
for subid in slidingWindows:
        windows_preop += slidingWindows[subid]["ses-preop"]["total"]
        windows_postop += slidingWindows[subid]["ses-postop"]["total"]
clustering_evaluate(windows_preop, range(2, 15), f"{save_path}/{time}_preop.png")
clustering_evaluate(windows_postop, range(2, 15), f"{save_path}/{time}_postop.png")
clustering_evaluate(windows_preop+windows_postop, range(2, 15), f"{save_path}/{time}_total.png")

In [3]:
with open("dFC/60_3dFCs.pkl", "rb") as f:
    dFCs = pickle.load(f)

In [4]:
# 全体拼接后聚类，输出状态变化
save_path = "states/total"
os.makedirs(save_path, exist_ok=True)
time = 60
dfcs_preop = None
dfcs_postop = None
for subid in dFCs:
    # preop
    if "total" in dFCs[subid]["ses-preop"]:
        if dfcs_preop is None:
            dfcs_preop = dFCs[subid]["ses-preop"]["total"].reshape((dFCs[subid]["ses-preop"]["total"].shape[0], 13456))
        else:
            dfcs_preop = np.vstack((dfcs_preop, dFCs[subid]["ses-preop"]["total"].reshape((dFCs[subid]["ses-preop"]["total"].shape[0], 13456))))
    else:
        for run in dFCs[subid]["ses-preop"]:
            if dfcs_preop is None:
                dfcs_preop = dFCs[subid]["ses-preop"][run].reshape((dFCs[subid]["ses-preop"][run].shape[0], 13456))
            else:
                dfcs_preop = np.vstack((dfcs_preop, dFCs[subid]["ses-preop"][run].reshape((dFCs[subid]["ses-preop"][run].shape[0], 13456))))
    # postop
    if "total" in dFCs[subid]["ses-postop"]:
        if dfcs_postop is None:
            dfcs_postop = dFCs[subid]["ses-postop"]["total"].reshape((dFCs[subid]["ses-postop"]["total"].shape[0], 13456))
        else:
            dfcs_postop = np.vstack((dfcs_postop, dFCs[subid]["ses-postop"]["total"].reshape((dFCs[subid]["ses-postop"]["total"].shape[0], 13456))))
    else:
        for run in dFCs[subid]["ses-postop"]:
            if dfcs_postop is None:
                dfcs_postop = dFCs[subid]["ses-postop"][run].reshape((dFCs[subid]["ses-postop"][run].shape[0], 13456))
            else:
                dfcs_postop = np.vstack((dfcs_postop, dFCs[subid]["ses-postop"][run].reshape((dFCs[subid]["ses-postop"][run].shape[0], 13456))))

# 释放内存
del dFCs
fcs = np.vstack((dfcs_preop, dfcs_postop))
# 释放内存，懒得改上面的
del dfcs_preop
del dfcs_postop

# 保存聚类对象
for k in target_states:
    # if k == 3:
    #     continue
    km = cluster.KMeans(k)
    km.fit(fcs)
    save_dir = f"{save_path}/cluster"
    os.makedirs(save_dir, exist_ok=True)
    with open(f"{save_dir}/km_{time}s_{k}states.pkl", "wb") as f:
        pickle.dump(km, f)
